In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import pickle
import numpy as np
from modules.bayes_optimization import BayesOpt, negUCB, negExpImprove
from modules.OnlineGP import OGP
import numpy as np
import importlib
import time
import glob
%matplotlib nbagg
# %matplotlib
# %matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import clear_output
import sys
import os
sys.path.insert(1, '/home/chenyu/Desktop/GaussianProcess/GPTrelated')
saveResultsQ = False
import scipy
import tensorflow as tf

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
os.environ["CUDA_VISIBLE_DEVICES"]="0" # specify which GPU to use
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    print(e)

In [23]:
# free space from GPU device
from numba import cuda
cuda.select_device(0)
cuda.close()

In [3]:
from keras import applications, optimizers, callbacks
from keras.models import Sequential, load_model
from keras.layers import Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [4]:
# model_path = 'CNNmodels/VGG16_coarse_linear_highCs_3fold_C5_negC1_WhiteNoise_newData_noFeatureCenter.h5'
# model_path = 'CNNmodels/VGG16_coarse_linear_highCs_3fold_C5_negC1_WhiteNoise_newData_smallShift.h5'
# model_path = 'CNNmodels/VGG16_coarse_linear_highCs_3fold_C5_negC1_WhiteNoise_newData.h5'
# model_path = 'CNNmodels/VGG16_coarse_linear_highCs_3fold_C5_negC1_WhiteNoise.h5'
# model_path = 'CNNmodels/VGG16_coarse_linear_highCs_3fold_C5_WhiteNoise.h5'
model_path = 'CNNmodels/VGG16_usim_40mradEmit_noApt.h5'
# model_path = 'CNNmodels/VGG16_30mrademit+defocus_30mradApt_lowAbr_test12_05.h5' 

In [5]:
model = applications.VGG16(weights=None, include_top=False, input_shape=(128, 128, 3))
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dense(1,activation=None))
new_model = Sequential()

for l in model.layers:
    new_model.add(l)

new_model.add(top_model)
new_model.load_weights(model_path)

In [6]:
def aperture_generator(px_size, simdim, ap_size):
    x = np.linspace(-simdim, simdim, px_size)
    y = np.linspace(-simdim, simdim, px_size)
    xv, yv = np.meshgrid(x, y)
    apt_mask = mask = np.sqrt(xv*xv + yv*yv) < ap_size # aperture mask
    return apt_mask

In [7]:
def scale_range(input, min, max):
    input += -(np.min(input))
    input /= np.max(input) / (max - min)
    input += min
    return input

In [8]:
abr_list = ["C10", "C12.x", "C12.y", "C21.x", "C21.y", "C23.x", "C23.y", "C30", 
"C32.x", "C32.y", "C34.x", "C34.y"]

abr_lim = [2e-6, 2e-6, 2e-6, 3e-5, 3e-5, 3e-5, 3e-5, 4e-4, 3e-4, 3e-4, 3e-4, 3e-4]

In [55]:
abr_coeff = 'C21'
file_list = glob.glob('NionRelated/'+ abr_coeff +'*50ms_bin1.npy')
print(file_list)

['NionRelated/C21.y_3e-05m_100steps_50ms_bin1.npy', 'NionRelated/C21.x_3e-05m_100steps_50ms_bin1.npy']


In [56]:
# ronch_list = np.load('NionRelated/C12.y_2e-06m_100steps_10ms_bin8.npy')
nsteps = 100
x_list = []
y_list = []
t_list = []
coeff_list = []

for file in file_list:
    name_list = file.replace('/','_').split('_')
    print(file, name_list)
    abr_coeff = name_list[1]
    idx = abr_list.index(abr_coeff)
    abr_range = abr_lim[idx]
    value_list = [(i - nsteps//2) * abr_range / nsteps for i in range(nsteps)]
    ronch_list = np.load(file)
    frame_list = []
#     print(abr_range, value_list)
    
    for i in range(ronch_list.shape[0]):
        x = [0.5 for i in range(len(abr_list))]
#         if value_list[i] == 0:
#             continue
        x[idx] = (value_list[i] + abr_range / 2) / abr_range
        x_list.append(x)
        frame = ronch_list[i,:,:]
        frame = scale_range(frame, 0, 1)
#         frame = frame * aperture_generator(128, 50, 40)
        new_channel = np.zeros(frame.shape)
        img_stack = np.dstack((frame, new_channel, new_channel))
        frame_list.append(img_stack)
    frame_list = np.concatenate([arr[np.newaxis] for arr in frame_list])
    prediction = new_model.predict(frame_list, batch_size = 1)
    y_list.append((1 - prediction[:,0])[:])
    t_list.append(name_list[-2])
    coeff_list.append(name_list[1])

NionRelated/C21.y_3e-05m_100steps_50ms_bin1.npy ['NionRelated', 'C21.y', '3e-05m', '100steps', '50ms', 'bin1.npy']
NionRelated/C21.x_3e-05m_100steps_50ms_bin1.npy ['NionRelated', 'C21.x', '3e-05m', '100steps', '50ms', 'bin1.npy']


In [1]:
fig, ax = plt.subplots(1,1, figsize = [8,6])
for i in range(len(t_list)):
#     p = ax.plot(np.array(value_list)[np.where(np.array(value_list) != 0)]*1e6, 1-y_list[i], linewidth = 2, 
#                label = t_list[i])
    p = ax.plot(np.array(value_list) * 1e6, 1 - y_list[i], linewidth = 2, label = coeff_list[i])
ax.set_xlabel(abr_coeff + ' (um)',fontsize = 16)
ax.set_ylabel('Normalized Defocus + Emit', fontsize = 16)
ax.tick_params(axis='x', labelsize=16)
ax.tick_params(axis='y', labelsize=16)
ax.legend(fontsize = 16)

NameError: name 'plt' is not defined

In [182]:
plt.imshow(ronch_list[48])

<IPython.core.display.Javascript object>

In [12]:
y_list = np.ndarray.flatten(np.asarray(y_list))
print(y_list.shape)

(1188,)


In [13]:
x_list = np.asarray(x_list)
print(x_list.shape)

(1188, 12)


In [14]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import time
import warnings
import numpy as np
warnings.filterwarnings("ignore")

In [15]:
# combine the data from linescans and the data from a test GP run

X = pd.DataFrame(x_list)
Y = pd.DataFrame(y_list).squeeze()

print('Y.shape',Y.shape,'\nX.shape',X.shape)

Y.shape (1188,) 
X.shape (1188, 12)


In [16]:
from Modules.tfgp_trainer import tfgp_trainer
tf.compat.v1.disable_eager_execution()

In [17]:
Niter = 2000

t0 = time.time()
tfgp = tfgp_trainer(X, Y,verbose=0)

tfgp.train(lr=0.005, 
           niter=Niter,
           optimizer=[tf.compat.v1.train.AdamOptimizer, tf.optimizers.SGD, tf.optimizers.RMSprop][0],
           gradient_clipping=0,
           kernel=['rbf','matern32','matern52'][0],
           correlations = False,
           offset = False,
           monitor_period=None,
           monitor_gradient=None) 

tfgp_t = time.time()-t0
print ('took ',tfgp_t ,' seconds')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


100% (2000 of 2000) |####################| Elapsed Time: 0:05:26 Time:  0:05:26


took  327.22573494911194  seconds


In [18]:
from Modules.utils import GP_plot_results
tfgp_results = GP_plot_results(tfgp,Y)

<IPython.core.display.Javascript object>

logLiklihod 3213.651596421564 
 amp 0.38758060127734106 
 noise 9.311000847597244e-05 
 ls [0.65420937 0.45567443 1.01362494 0.3303296  0.50292911 0.35046496
 0.34078097 0.39508727 1.8988137  2.54367039 0.52326547 0.60859243]


In [19]:
from Modules.sk_trainer import sk_trainer
sk=sk_trainer()
sk.sk_train(X, Y, kernels=['rbf'], offset=False, verboseQ=True, saveQ=False)

********* 
 rbf
took  195.3130910396576  seconds
Inital kernel: 3.16**2 * RBF(length_scale=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) + WhiteKernel(noise_level=0.01)
Learned kernel: 0.558**2 * RBF(length_scale=[0.604, 0.124, 0.892, 0.21, 0.47, 0.0906, 0.209, 0.252, 1.76, 2.27, 0.494, 0.578]) + WhiteKernel(noise_level=0.000135)
Log-marginal-likelihood: 3319.669


In [20]:
from Modules.gpy_trainer import gpy_trainer
gpy=gpy_trainer()
gpy.gpy_train(X, Y[:,None], kernels= ['rbf'], max_iters=5000, offset=False, verboseQ=True, saveQ=False)

********* 
 rbf
Optimization restart 1/5, f = -3328.633205097573
Optimization restart 2/5, f = -3269.187671264865
Optimization restart 3/5, f = -3329.607337939866
Optimization restart 4/5, f = -3217.926980250713
Optimization restart 5/5, f = -3362.82745192506
took  64.75490403175354  seconds
Log-marginal-likelihood: 3362.827


In [21]:
# collect the results into a pandas dataframe to display
pd.DataFrame({"Param": ['time','negative log likelihood', 'amplitude', 'noise', 'offset'],
              "tfgp": [tfgp_t, tfgp_results['ll'], tfgp_results['amp_param'], tfgp_results['noise_param_variance'] , tfgp_results['offset_param']],
              "SK": [sk.sk_t, sk.results['ll'], sk.results['amp_param'], sk.results['noise_param_variance'] , sk.results['offset_param']],
              "GPy": [gpy.gpy_t, gpy.results['ll'], gpy.results['amp_param'], gpy.results['noise_param_variance'] , gpy.results['offset_param']],
             })


Param         tfgp           SK          GPy
0                     time   327.225735   195.313091    64.754904
1  negative log likelihood -3213.651596  3319.668927  3362.827452
2                amplitude     0.387581     0.311160     0.321767
3                    noise     0.000093     0.000135     0.000108
4                   offset          NaN          NaN          NaN

In [22]:
print('Lengthscales Comparison')
pd.DataFrame({"": X.keys(),
#               "Param":['HP','HP diff','S6','S7','OBJ','S3','S4'],
              "tfgp": tfgp_results['length_scale_param'],
              "SK": sk.results['length_scale_param'] ,
              "GPy": gpy.results['length_scale_param']
             })

Lengthscales Comparison


tfgp        SK       GPy
0    0  0.654209  0.604221  0.577324
1    1  0.455674  0.124074  0.068702
2    2  1.013625  0.892121  0.200620
3    3  0.330330  0.210202  0.204559
4    4  0.502929  0.470003  0.453899
5    5  0.350465  0.090618  0.087588
6    6  0.340781  0.209209  0.199604
7    7  0.395087  0.252070  0.252531
8    8  1.898814  1.762816  0.516366
9    9  2.543670  2.265491  1.545480
10  10  0.523265  0.494262  0.483063
11  11  0.608592  0.578155  0.259449